# Module 1: Feature Spaces and Collaborative Filtering

## Objective

Companies like Bol.com, Amazon, Google, and Netflix want to predict which products (books, movies, etcetera) you might find interesting. In this assignment, you will create a system capable of making such predictions.

Such predictions are often powered by closely examining your behavior on a website. Users view products, make purchases, and provide ratings. All those pieces of information provide insight into the kinds of products you like. The next step is to find similar products to those you already like, as there's a high chance you will also enjoy them.

One way to determine if, for example, two movies are similar is to compare their content. (For instance: Do they cover the same topic? Are some actors in both?) Recommending movies based on such comparisons is known as _content-based filtering_, which we will explore in a later assignment.

This modules focuses on _collaborative filtering_.  The main idea here is that the collective behavior of users provides information about the products.

In collaborative filtering, you can distinguish between two strategies: _user-based filtering_ and _item-based filtering_. User-based filtering aims to discover to what extent users have similar behavior and recommend a movie based on that. On the other hand, item-based filtering seeks to determine how similar two movies are to each other.

Which approach is the best? Content-based filtering, user-based collaborative filtering, or item-based collaborative filtering? There is no definitive answer to this question. The most suitable approach depends heavily depends on the specific application.

Moreover, each approach involves many subtle choices that can influence performance. For instance, you can make different choices in measuring the similarity between two products (or users). Two commonly used measures are _Euclidean similarity_ and _cosine similarity_. In practice, there are even more options available.

## Structure

In this module, you will work on implementing _user-based filtering_ and _item-based filtering_. In the next module, you will compare the performances of these approaches. 

Contents:

- [Part 1: Item-based Filtering](#Part-1:-Item-based-Filtering): Recommend movies to users based on the similarity between movies. 
    - [Reading Data](#Reading-Data): Load and parse the MovieLens data. Questions: [Question 1](#Question-1)
    - [Feature Spaces](#Feature-Spaces): Represent the data spatially. Questions: [Question 2](#Question-2), [Question 3](#Question-3), [Question 4](#Question-4), [Question 5](#Question-5), [Question 6](#Question-6), [Question 7](#Question-7),
    - [Similarity](#Similarity) and [Missing Values](#Missing-Values): How do we determine similarity between modies? Questions: [Question 8](#Question-8), [Question 9](#Question-9)
        - [Euclidean similarity](#Euclidean-similarity): The easiest way. Questions: [Question 10](#Question-10)
        - [Cosine similarity](#Cosine-similarity): A better way. Questions: [Question 11](#Question-11).
        - [Intermezzo](#Intermezzo---Difference-in-Computation-Time): Make your algorithm run fast enough by using built-in functions.
        - [Mean Centering](#Mean-Centering): Account for indivdual biasses in rating movies. Questions: [Question 12](#Question-12)
    - [Predictions (K-nearest neighbor)](#Predictions-(K-nearest-neighbor)): The main algorithm. Questions: [Question 13](#Question-13), [Question 14](#Question-14), [Question 15](#Question-15), [Question 16](#Question-16)
- [Part 2: User-Based Filtering](#Part-2:-User-Based-Filtering): Recommend movies to users based on the similarity between users. 
    - Questions: [Question 17](#Question-17), [Question 18](#Question-18), [Question 19](#Question-19), [Question 20](#Question-20), [Question 21](#Question-21), [Question 22](#Question-22), [Question 23](#Question-23)

## Study Material

- [Collaborative filtering, Harvard CS50](https://www.youtube.com/watch?v=Eeg1DEeWUjA)
- [Recommender Systems: The Textbook](https://link.springer.com/book/10.1007/978-3-319-29659-3) (free download from UvA network) Chapter 1 and 2.

To build a collaborative filtering system, we will use a dataset created by [MovieLens](http://movielens.org), a movie recommendation website (already included in the assignment). The data is stored in the files `movies.csv`, `ratings.csv`, and `tags.csv`. These are all [comma-separated value](http://en.wikipedia.org/wiki/Comma-separated_values) files.

We want to utilize this data to recommend movies to users that they haven't watched yet. This will be based on ratings they have given to _other_ movies. Therefore, we want to know how well a user will _likely_ enjoy a particular movie. If we reframe the problem in terms of ratings: **Given a user who hasn't watched a certain movie, _predict the rating_ the user would likely give to the movie after watching it.**

## Getting Started

Let's start by loading the libraries we'll need:

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
np.seterr(divide='raise', over='raise', under='raise', invalid='raise')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
from typing import Optional

import answers

# Part 1: Item-based Filtering

Our end goal is to recommend a movie to a user. There are various ways to achieve that. In this assignment, we will first apply collaborative filtering. The idea behind collaborative filtering is to leverage user interactions to recommend items to users. In the MovieLens dataset, these interactions are represented by ratings.

First, let's focus on _item-based filtering_. The intuition is simple: if a user gives the same rating to two different movies, it indicates that those movies are similar. By considering all ratings from all users, we can get a reasonably complete picture of the _similarity_ between two movies.

This part consists of the following steps:
1. Reading the data
2. Transforming the data into a usable format (utility matrix)
3. Determining similarities
4. Making a recommendation

### Reading Data

We'll start with a micro-dataset. This small set contains 3 movies and 5 users. It is a subset of the MovieLens dataset, which allows you to easily test your code since you have an overview of all the data. For comparison, the original dataset contains 27,000 movies and 138,000 users. The algorithms we are going to apply should, in principle, also work on the full dataset, although we will experience some challenges and hardships in the next module.

The dataset consists of 3 tables (`DataFrame`s):
- **movies**, a table with the movie's id, name, and genre.
- **users**, a table with the user's id and name.
- **ratings**, a table with the user's id, the movie's id, the corresponding rating, and the timestamp.

In the following tasks, we will only use the ratings table. The other two tables are provided for our intuition and reasoning. It's easier to talk about Shrek than movie 4306, so please do so when asked about certain movies in open questions.

Load the data:

In [ ]:
# Load the micro movielens set. This set contains 5 users and 3 movies

movies = pd.read_csv('./micro-movielens/movies.csv', index_col='movieId')
users = pd.read_csv('./micro-movielens/users.csv', index_col='userId')
ratings = pd.read_csv('./micro-movielens/ratings.csv')

display(movies)
display(users)
display(ratings)

### Question 1

\[1 pt.\]

In the algorithm we are going to create, we often need to retrieve the rating for a specific combination of movie and user. So it's useful to create a function that does this for us. The function `get_rating()` is defined below. Note that it's not yet implemented. The function should retrieve the rating a user gave to a specific movie from the ratings table. For example, `get_rating(ratings, 182, 2571)` should return `5.0`. Special case: if there is no rating for the combination of movie and user, `get_rating()` should return the value `NaN`. You can use `np.nan` for this purpose.

Implement `get_rating()` below:

In [ ]:
# Define a helper function for accessing data.

def get_rating(ratings: pd.DataFrame, userId: int, movieId: int) -> float:
    """
    Given a userId and movieId, this function returns the corresponding rating.
    Should return NaN if no rating exists.
    """
    
    # TODO
    
    
# test. This should print:
# The rating for user 182 and movie 2710 is 4.5
# The rating for user 204 and movie 4306 is nan

rating1 = get_rating(ratings, 182, 2710)
print(f'The rating for user 182 and movie 2710 is {rating1}')
rating2 = get_rating(ratings, 204, 4306)
print(f'The rating for user 204 and movie 4306 is {rating2}')

### Check

In [ ]:
answers.test_1(get_rating, ratings)

### Feature Spaces

For analyzing the data, we can generate a table that lets us easily see how users rated the movies. Like this table:

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th></th><th>182</th><th>195</th><th>204</th><th>376</th><th>542</th></tr></thead><tbody><tr><th>2571</th><td>5.0</td><td>3.0</td><td>4.5</td><td>3.5</td><td>5.0</td></tr><tr><th>2710</th><td>4.5</td><td>1.0</td><td>5.0</td><td>1.5</td><td>0.5</td></tr><tr><th>4306</th><td>4.0</td><td>3.0</td><td>NaN</td><td>4.0</td><td>5.0</td></tr></tbody></table>

Such a table is called the **utility matrix**. In this matrix, it is easy to see that user `542` (Nicolai) rated the movie `2710` (The Blair Witch Project) quite poorly, with a rating of `0.5`.

We are working on item-based filtering, so we want to compare movies with each other. It is a convention to put the movies on the vertical axis. Thus, the rows are the _items_ we want to compare with each other, and the columns are the **features** of the items (in this case, the ratings from the users). So here each movie has five features: the rating from each user.

### Question 2

\[2 pt.\]

The function `pivot_ratings()` below is not complete yet and should generate the table mentioned above. You need to write the function yourself; this means you are not allowed to use the built-in `pivot()` function of pandas (yet). Special case: if there is no rating for the combination of movie and user, `NaN` should be placed in that position in the table. You can use `np.nan` for this purpose.

Tip 1: You can use the `get_rating()` function you defined in the previous question.

Tip 2: The test expects the output of the function to be a `DataFrame` with elements of type `float`. You can use the argument `dtype=float` when creating a new `DataFrame` to ensure this.

Implement `pivot_ratings()` below:

In [ ]:
def pivot_ratings(ratings: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a rating table as input and computes the utility matrix.
    """
    # TODO
    

# test
utility_matrix = pivot_ratings(ratings)
display(utility_matrix)

### Check

In [ ]:
answers.test_2(pivot_ratings, ratings)

---

### 🏁 Checkpoint - Pivot Function in Pandas

From this point on, we will use the built-in pivot function of pandas. This function is written in C and is most likely faster than anything you could come up with on your own. 
> This also means that if the previous task was not successful, you can continue with the assignment now.

In [ ]:
def pivot_ratings(ratings: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a rating table as input and computes the utility matrix using pandas pivot function.
    Overwrites the pivot function from the previous exercise.
    """
    
    return ratings.pivot(index = 'movieId', columns = 'userId', values = 'rating') 

utility_matrix = pivot_ratings(ratings)
display(utility_matrix)

The utility matrix provides us with a lot of information about the relationship between movies and users. We can visualize this data to gain insights. Let's start by creating some histograms:

In [ ]:
# Create histograms for movie ratings.
hist = utility_matrix.T.hist(bins=[0,1,2,3,4,5], layout=(1,5), figsize=(14,2));

# set axes
for ax in hist[0]:
    ax.set_ylim(0, 3.1)
    ax.set_xlim(-0.1, 5.1)

### Question 3

\[2 pt.\]

Based on the histograms above: what is the _name_ of the movie with the most divided opinions (highest variance in ratings)?

YOUR ANSWER HERE

The histograms do not provide much detailed information about the movies. By using scatter plots, we can make the data more insightful.

In [ ]:
def plot_scatter(matrix: pd.DataFrame, feature1: str, feature2: str, 
                 colors: list[str] = ['#e41a1c', '#377eb8', '#4eae4b', '#994fa1', '#ff8101']) -> None:
    # setup plot
    elements = matrix.shape[0]
    scatter = matrix.plot.scatter(feature1, feature2, c = colors[:elements])
    scatter.set_title('movie rating plot')
    
    # plot settings
    scatter.set_xlim(-0.1, 5.1)
    scatter.set_ylim(-0.1, 5.1)
    scatter.set_xlabel('rating for userId {}'.format(feature1))
    scatter.set_ylabel('rating for userId {}'.format(feature2))
    scatter.legend([mpatches.Circle((0,0), 1, fc=c) for c in colors], matrix.index, loc = 'lower right', title = 'movieId')
    
# scatter plot    
plot_scatter(utility_matrix, 542, 182)

In the above plot, the points represent the three movies. The x-axis represents Nicolai's (542) rating, and the y-axis represents Gotlob's (182) rating. We can imagine the movies as points in an abstract space. Such a space is called a **feature space**.

### Question 4

\[2 pt.\]

When looking at this space, which two movies do you think are most similar? Why do you think so?

YOUR ANSWER HERE

### Question 5

\[1 pt.\]

Plot the feature space as above, but this time for Isaac and Gotlob.

Tip: If you quickly want to check which userId corresponds to which name, you can use the command `display(users)`.

In [ ]:
# TODO


### Question 6

\[1 pt.\]

Why is the plot showing only two movies?

YOUR ANSWER HERE

By representing the ratings as a space (a feature space), you can see if two movies are physically close. The physical distance can provide insights into whether they might be similar. In other words, if many users who liked The Matrix also gave a high rating to The Blair Witch Project, these movies might be similar. A recommender system can use this information to recommend The Blair Witch Project to someone who liked The Matrix.

In the above examples, we used a 2D space, with one user per axis. As you saw, the outcome can vary significantly depending on the space used. Normally, you don't use just a 2D feature space, but the number of dimensions is equal to the number of features in the table. In our example, there are five dimensions (namely, the ratings of the five users, Ada, Nicolai, Isaac, Gotlob, and Carl).

Unfortunately, plotting a high-dimensional feature space is challenging. With some effort, you can still plot three dimensions:

In [ ]:
def plot3d(matrix: pd.DataFrame, feature1: str, feature2: str, feature3: str, 
           colors: list[str] = ['#e41a1c', '#377eb8', '#4eae4b', '#994fa1', '#ff8101']) -> None:  
    # setup 3d plot
    fig = plt.figure()
    ax = Axes3D(fig, auto_add_to_figure=False)
    fig.add_axes(ax)
    
    # enter data
    for i, ind in enumerate(matrix.index):
        scatter = ax.scatter(matrix.loc[ind, feature1], matrix.loc[ind, feature2], matrix.loc[ind, feature3], c = colors[i])

    # plot settings
    ax.legend([mpatches.Circle((0,0), 1, fc=c) for c in colors], matrix.index, loc = 'lower right', title = 'movieId')
    ax.set_xlabel('rating for userId {}'.format(feature1))
    ax.set_ylabel('rating for userId {}'.format(feature2))
    ax.set_zlabel('rating for userId {}'.format(feature3))
    
    plt.show()

plot3d(utility_matrix, 376, 182, 542)

Unfortunately, such a 3D plot is hard to read. Just how far apart are the movies in the above plot? Moreover, we have more dimensions that we want to visualize. A common way to plot such a higher-dimensional space is a _scatter plot matrix_. In this matrix, we create scatter plots of all combinations of dimensions, which works much better:

In [ ]:
def plot_scatter_matrix(matrix: pd.DataFrame, 
                        colors: list[str] = ['#e41a1c', '#377eb8', '#4eae4b', '#994fa1', '#ff8101', '#000000', '#999999'],
                        figsize: tuple[int, int] = (8,8)) -> None:
    # setup plot
    max_val = matrix.max(axis = 1).max()
    min_val = matrix.min(axis = 1).min()
    fmatrix = matrix.fillna(min_val - 1)
    elements = fmatrix.shape[0]
    axarr = pd.plotting.scatter_matrix(fmatrix, s = 180.0, c = colors[:elements], figsize=figsize, alpha = 1.0, diagonal = '')
    m = fmatrix.shape[1]
    max_val = matrix.max(axis = 1).max()
    min_val = matrix.min(axis = 1).min()
    
    # set axes for all subplots
    for i in range(m):
        for j in range(m):
            axarr[i,j].set_xlim(min_val - 0.5, max_val + 0.5)
            axarr[i,j].set_ylim(min_val - 0.5, max_val + 0.5)

    # set labels for subplots
    labels = fmatrix.index 
    axarr[0,0].legend([mpatches.Circle((0,0),1,fc=c) for c in colors], labels, loc = 'lower left');

plot_scatter_matrix(utility_matrix)

### Question 7

\[3 pt.\]

a) When looking at the above scatter plot matrix, which two movies do you think are most similar? Provide an argument.

b) Why are there no plots drawn on the diagonal?

c) Why do some plots contain three points and others two?

YOUR ANSWER HERE

### Similarity

In the previous part, we looked at whether we could see which movies are most similar to each other based on the plots of the data. Now, we will try to formalize those intuitions.

Ultimately, we want to create a function that, based on the given data, can calculate the extent to which two movies are similar.

First, we need a good definition of distance. The simplest way is to look at the differences in the ratings the users gave. This table provides an example:

<table><thead><tr style="text-align: center;"><th></th><th>2571</th><th></th><th>2710</th><th></th><th>difference</th><th></th></tr></thead>
<tbody><tr><th>182</th><td>5.0</td><td>-</td><td>4.5</td><td>=</td><td>0.5</td><td>(Gotlob likes The Matrix more than The Blair Witch Project)</td></tr>
<tr><th>195</th><td>3.0</td><td>-</td><td>1.0</td><td>=</td><td>2.0</td><td>(Ada likes Blair Witch Project much less)</td></tr>
<tr><th>204</th><td>4.5</td><td>-</td><td>5.0</td><td>=</td><td>-0.5</td><td>(Isaac likes The Matrix slightly less)</td></tr>
<tr><th>376</th><td>3.5</td><td>-</td><td>1.5</td><td>=</td><td>2.0</td><td>(Carl likes The Blair Witch Project much less)</td></tr>
<tr><th>542</th><td>5.0</td><td>-</td><td>0.5</td><td>=</td><td>4.5</td><td>(Nicolai dislikes The Blair Witch Project much more than The Matrix)</td></tr>
</tbody></table>

Next, we calculate the distance between the two movies by taking the sum of the absolute values of these differences:

$$
d = {\rm abs}(0.5) + {\rm abs}(2.0) + {\rm abs}(-0.5) + {\rm abs}(2.0) + {\rm abs}(4.5) = 10.5
$$

This distance measure is called the **Manhattan distance** (look [here](https://en.wikipedia.org/wiki/Taxicab_geometry) if you want to know why it's called that). Note that this is a distance measure: the value becomes _larger_ as the movies are _less_ similar. In practice, we want a similarity measure: a value that becomes _larger_ as the movies are _more_ similar. Converting a distance measure to a similarity measure is quite straightforward. There are several ways to do this, but a commonly used function is:

$$
s = \frac{1}{1 + d}
$$

Where $s$ stands for _similarity_ and $d$ for the _distance_. In the case of this example, the similarity is:

$$
\frac{1}{1 + 10.5} = 0.087
$$

For completeness, we also provide the formal definition of the Manhattan distance. Given item (movie) $x$ and item $y$ with $n$ features (user ratings), we define the Manhattan distance $d(x, y)$ as follows:

$$
d(x, y) = \Sigma_{i=1}^{n}|x_{i} - y_{i}|
$$ 

$$
s(x, y) = \frac{1}{1 + d(x, y)}
$$

### Missing Values

Let's see what happens in practice when we compare Shrek with The Matrix:


<table><thead><tr style="text-align: center;"><th></th><th>2571</th><th></th><th>4306</th><th></th><th>difference</th><th></th></tr></thead>
<tbody><tr><th>182</th><td>5.0</td><td>-</td><td>4.0</td><td>=</td><td>1.0</td><td>(Gotlob likes The Matrix more than Shrek)</td></tr>
<tr><th>195</th><td>3.0</td><td>-</td><td>3.0</td><td>=</td><td>0.0</td><td>(Ada likes both movies equally)</td></tr>
<tr><th>204</th><td>4.5</td><td>-</td><td><b>NaN</b></td><td>=</td><td><b>NaN</b></td><td>???</td></tr>
<tr><th>376</th><td>3.5</td><td>-</td><td>4.0</td><td>=</td><td>-0.5</td><td>(Carl likes Shrek slightly more)</td></tr>
<tr><th>542</th><td>5.0</td><td>-</td><td>5.0</td><td>=</td><td>0.0</td><td>(Nicolai likes both movies equally)</td></tr>
</tbody></table>


$$
d = {\rm abs}(1.0) + {\rm abs}(0.0) + {\rm abs}({\bf NaN}) + {\rm abs}(-0.5) + {\rm abs}(0.0) = {\bf NaN}
$$

$$
s = \frac{1}{1 + d} = \frac{1}{1 + {\bf NaN}} = {\bf NaN}
$$

Because the column for Shrek contains a NaN value, the result is NaN everywhere. This is a problem. It means that we cannot determine the similarity between The Matrix and Shrek. This is the case even when there is only one missing value. So, if we have a movie in the dataset that one user has not seen, we cannot calculate the similarity for that movie. When dealing with larger datasets, this problem becomes more significant: there will always be someone who has not seen a particular movie.

The solution is to fill in something before calculating the similarity. For example, you could replace all NaNs with 0. This is simple, but not very useful. It assumes that if someone hasn't seen the movie, that person finds the movie worthless. That is probably not a fair assumption.

A better approximation is to calculate the average rating for the movie and use that to fill in all NaN values. For example, for Shrek, we take the average of all known values:

$$
\frac{4.0 + 3.0 + 4.0 + 5.0}{4} = {\bf 4.0}
$$

And we use that to fill in the NaN value:

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th></th><th>182</th><th>195</th><th>204</th><th>376</th><th>542</th></tr></thead><tbody><tr><th>2571</th><td>5.0</td><td>3.0</td><td>4.5</td><td>3.5</td><td>5.0</td></tr><tr><th>2710</th><td>4.5</td><td>1.0</td><td>5.0</td><td>1.5</td><td>0.5</td></tr><tr><th>4306</th><td>4.0</td><td>3.0</td><td><b>4.0</b></td><td>4.0</td><td>5.0</td></tr></tbody></table>

The `fill_nan_mean()` function below replaces the NaN values with the mean of the respective row.

In [ ]:
def fill_nan_mean(matrix: pd.DataFrame) -> pd.DataFrame:
    # Get the mean for each row
    meanv = matrix.mean(axis = 1, skipna=True)
    
    # Fill all nan values with mean 
    # (fillna only works for collumns, so the matrix is transposed twice)
    filled1 = matrix.T.fillna(meanv).T
    
    # If there's still nan's left, it means the entire row is nan, so
    # no mean could be computed. fill with 0 in that case
    filled2 = filled1.fillna(0)
    return filled2
    
utility_matrix_filled = fill_nan_mean(utility_matrix)
display(utility_matrix_filled)

In [ ]:
def manhattan_distance(series1: pd.Series, series2: pd.Series) -> pd.Series:
    """
    Compute manhattan distance between two datapoints.
    """     
    # Compute the distances for the features
    distances = series1 - series2
    
    # Return the absolute sum
    return distances.abs().sum()

def manhattan_similarity(matrix: pd.DataFrame, id1: int, id2: int) -> Optional[float]:
    """
    Compute manhattan similarity between two datapoints. Can return a nan-value when there are no shared features.
    """
    # compute distance
    distance = manhattan_distance(matrix.loc[id1], matrix.loc[id2])
    
    # else return similarity
    return 1 / (1 + distance)

similarity = manhattan_similarity(utility_matrix_filled, 2571, 4306)
print(f'Similarity between The Matrix and Shrek: {similarity:.2f}')

Just like before, where we created the utility matrix to easily look up ratings, we will now create a matrix to easily look up similarities:

<table border="1" class="dataframe"><thead><tr style="text-align: right;"><th></th><th>2571</th><th>2710</th><th>4306</th></tr></thead><tbody><tr><th>2571</th><td>1.000000</td><td>0.095238</td><td>0.400000</td></tr><tr><th>2710</th><td>0.095238</td><td>1.000000</td><td>0.095238</td></tr><tr><th>4306</th><td>0.400000</td><td>0.095238</td><td>1.000000</td></tr></tbody></table>

### Question 8

\[2 pt.\]

The function `create_similarity_matrix_manhattan()` below is not complete. The matrix with the correct dimensions is already generated, but the rest is up to you. Fill the matrix with the correct similarity values:

In [ ]:
def create_similarity_matrix_manhattan(matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Creates the similarity matrix based on manhattan distance.
    """
    
    similarity_matrix = pd.DataFrame(data=0, index=matrix.index, columns=matrix.index, dtype=float)
    
    # TODO
    
    
    return similarity_matrix
           
similarity_matrix_manhattan = create_similarity_matrix_manhattan(utility_matrix)
display(similarity_matrix_manhattan)

### Check

In [ ]:
answers.test_8(create_similarity_matrix_manhattan, utility_matrix)

### Question 9

\[1 pt.\]

In the above matrix, all cells on the diagonal contain the value 1.0. Why is that the case?

YOUR ANSWER HERE

### Euclidean similarity

Manhattan distance is an easy-to-use metric, but in practice it's not often used. Imagine you have two movies with similar ratings but with small differences each time. When you calculate the Manhattan distance, you sum up all the differences in each dimension, no matter how small. As a result, it seems like the movies are quite different.

A more popular metric is the **Euclidean distance**. This distance is derived when you treat the feature space as a regular physical space and use the Pythagorean theorem to determine the distance between two points (movies). The use of powers in the formula of this distance measure effectively disregards small differences.

Similar to the Manhattan distance, we first look at the differences in distance.

<table>
  <thead>
    <tr style="text-align: center;">
      <th></th>
      <th>2571</th>
      <th></th>
      <th>4306</th>
      <th></th>
      <th>difference</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>182</th>
      <td>5.0</td>
      <td>-</td>
      <td>4.0</td>
      <td>=</td>
      <td>1.0</td>
      <td>(Gotlob prefers The Matrix over Shrek)</td>
    </tr>
    <tr>
      <th>195</th>
      <td>3.0</td>
      <td>-</td>
      <td>3.0</td>
      <td>=</td>
      <td>0.0</td>
      <td>(Ada likes both movies equally)</td>
    </tr>
    <tr>
      <th>204</th>
      <td>4.5</td>
      <td>-</td>
      <td>4.0</td>
      <td>=</td>
      <td>0.5</td>
      <td>(Isaac slightly prefers The Matrix)</td>
    </tr>
    <tr>
      <th>376</th>
      <td>3.5</td>
      <td>-</td>
      <td>4.0</td>
      <td>=</td>
      <td>-0.5</td>
      <td>(Carl slightly prefers Shrek)</td>
    </tr>
    <tr>
      <th>542</th>
      <td>5.0</td>
      <td>-</td>
      <td>5.0</td>
      <td>=</td>
      <td>0.0</td>
      <td>(Nicolai likes both movies equally)</td>
    </tr>
  </tbody>
</table>

Next, we use the Pythagorean theorem to calculate the Euclidean distance. In a 2D space, the theorem is $d^2 = x^2 + y^2$, or rewritten: $d = \sqrt{x^2 + y^2}$. In the 3D case, it's $d = \sqrt{x^2 + y^2 + z^2}$. In general, for $n$ features $f_{1}$ to $f_{n}$:

$$
d = \sqrt{f_{1}^2 + f_{2}^2 + \ldots + f_{n}^2}
$$

In our example, the distance $d$ is:

$$
d = \sqrt{1.0^2 + 0.0^2 + 0.5^2 + -0.5^2 + 0.0^2} \approx 1.22
$$

If we convert this distance to similarities:

$$
1 / (1 + 1.22) \approx 0.45
$$

The formal definitions:

$$
d(x, y) = \sqrt{\Sigma_{i=1}^{n}(x_{i} - y_{i})^2}
$$

$$
s(x, y) = 1/(1 + d(x, y))
$$

### Question 10

\[1 pt.\]

Implement the function `create_similarity_matrix_euclid()` below:

In [ ]:
# TODO



def create_similarity_matrix_euclidean(matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Creates the similarity matrix based on eucledian distance.
    """
    
    similarity_matrix = pd.DataFrame(data=0, index=matrix.index, columns=matrix.index, dtype=float)
    
    # TODO
    
    
    return similarity_matrix

similarity_matrix_euclidean = create_similarity_matrix_euclidean(utility_matrix_filled)
display(similarity_matrix_euclidean)

### Check

In [ ]:
answers.test_10(create_similarity_matrix_euclidean, utility_matrix)

### Cosine similarity

In practice, _cosine similarity_ is often used for comparing different items. To determine this, we draw two imaginary lines from the origin to the data points we want to compare and measure the angle between these two lines:

<img src='./include/cosine similarity.png'>

In the example above, we calculate the cosine similarity for movies 2571 and 2710, considering only the ratings from users 195 and 542. The angle $\Theta$ between the two lines for both movies is $0.57$ degrees (rad). The cosine similarity is defined as the cosine of this angle. In this example, the similarity is approximately cos$(0.57) \approx 0.84$.

In the case of cosine similarity, the distance from a point to the origin doesn't matter. Why could this be an advantage?

Calculating cosine similarity is simpler than it might seem. For two points $A = (a_1, a_2)$ and $B = (b_1, b_2)$, the cosine similarity is:
$$
\textrm{cos}(A, B) = \frac{a_1*b_1 + a_2*b_2}{\sqrt{a_1^2 + a_2^2}*\sqrt{b_1^2 + b_2^2}}
$$

In our case, that becomes:
$$
\textrm{cos}(2571, 2710) = \frac{3.0*1.0 + 5.0*0.5}{\sqrt{3.0^2 + 5.0^2} * \sqrt{1.0^2 + 0.5^2}} \approx 0.84
$$

In the above calculation, we assumed for simplicity that there are only two features (the ratings from users 195 and 542). In reality, we have more than two features; in our case, we have five. The general formula for the cosine of items with $n$ features $A = (a_1, a_2, \ldots, a_n)$ and $B = (b_1, b_2, \ldots, b_n)$ is:

$$
\textrm{cos}(A, B) = \frac{a_1*b_1 + a_2*b_2 + \ldots + a_n*b_n}{\sqrt{a_1^2 + a_2^2 + \ldots + a_n^2}*\sqrt{b_1^2 + b_2^2 + \ldots b_n^2}}
$$

In our case, that becomes:
$$
\textrm{cos}(2571, 2710) = \frac{5.0*4.5  + 3.0*1.0 + 4.5*5.0 + 3.5*1.5 + 5.0*0.5}{\sqrt{5.0^2 + 3.0^2 + 4.5^2 + 3.5^2 + 5.0^2} * \sqrt{4.5 ^ 2  + 1.0^2 + 5.0^2 + 1.5^2 + 0.5^2}} \approx 0.83
$$

The formal definition:
$$
\textrm{cos}(A, B) = \frac{\sum_{i = 1}^n a_i * b_i}{\sqrt{\sum_{i = 1}^n a_i^2}{\sqrt{\sum_{i = 1}^n b_i^2}}}
$$

### Question 11

\[3 pts.\]

Use the formula above to implement the function `create_similarity_matrix_cosine_temp()` below. There is one important edge case to consider:

- If a row contains only `0`s (all ratings for a movies were `0`), division by zero can occur. Detect where this happens and handle it by placing a `0` in the new similarity matrix.

In [ ]:
# TODO


similarity_matrix_cosine = create_similarity_matrix_cosine_temp(utility_matrix_filled)
display(similarity_matrix_cosine)

### Check

In [ ]:
answers.test_11(create_similarity_matrix_cosine_temp, utility_matrix)

### 🏁 Checkpoint - Cosine Similarity from sklearn
Similarly to the pivot function: it's better to replace the previous function with one defined in a library. Below is a demonstration of how you can use the `cosine_similarity` function from the `sklearn` library.

This also means that you can proceed from here if implementing this function did not work out.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cosine_similarity_sklearn

def create_similarity_matrix_cosine(utility_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Creates similarity matrix from the utility matrix. The function from sklearn
    does not take NaN values so we temporarily fill them with the mean rating
    from a movie.
    """    
    # sklearn needs a numpy ndarray as input, so convert the matrix
    matrix = np.array(utility_matrix)
    
    similarities = cosine_similarity_sklearn(matrix)
    
    # Convert the output back to a pandas dataframe
    similarity_matrix = pd.DataFrame(data=similarities[0:,0:],
                                    index=utility_matrix.index,
                                    columns=utility_matrix.index)

    return similarity_matrix

similarity_matrix_cosine = create_similarity_matrix_cosine(utility_matrix_filled)
display(similarity_matrix_cosine)

#### Intermezzo - Difference in Computation Time
Of course, we can claim that a function from a library is faster, but actually showing the difference works better. Below, we use one of the `cellmagic` [commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) in a Jupyter Notebook: `%%timeit`.

In [ ]:
%%timeit -r 10
similarity_matrix_cosine = create_similarity_matrix_cosine_temp(utility_matrix_filled)

In [ ]:
%%timeit -r 10
similarity_matrix_cosine = create_similarity_matrix_cosine(utility_matrix_filled)

### Mean Centering

The issue with cosine similarity is that it's sensitive to bias. In this case, the average value of a feature not being 0 leads to bad scores when using cosine similarity. In our case, this is a problem because the average rating per user is certainly not 0.

The solution to this is **mean centering**. The idea here is to subtract the average rating of _the same_ user from each rating given by that user. Since in the utility matrix, each column represents the ratings of one user, this comes down to subtracting the average from each column.

Below, you can see the effect of this. In Figure 1, we see cosine similarity without mean centering and in Figure 2, we see the cosine similarity of the same two points but with centering applied. Applying cosine similarity to a mean-centered utility matrix is also known as the **adjusted cosine**.

<table>
    <tr style ="background-color: #ffffff"><td style="text-align:center">Figure 1</td><td style="text-align:center">Figure 2</td></tr>
    <tr><td><img src='./include/cosine similarity.png'></td><td><img src='./include/adjusted cosine similarity.png'></td></tr>
</table>

### Question 12

\[1 pt.\]

Implement the function `mean_center_columns()` below:

In [ ]:
def mean_center_columns(matrix: pd.DataFrame) -> pd.DataFrame:
    # TODO
    
    
centered_utility_matrix = mean_center_columns(utility_matrix_filled)
similarity = create_similarity_matrix_cosine(centered_utility_matrix)

print("The original utility matrix:")    
display(utility_matrix)

print("The matrix with filled NaN values:")
display(utility_matrix_filled)

print("The filled matrix mean centered:")
display(centered_utility_matrix)

print("The similarity matrix based on the mean centered utility matrix:")
display(similarity)

### Check

In [ ]:
answers.test_12(mean_center_columns, utility_matrix_filled, create_similarity_matrix_cosine)

### Predictions (K-nearest neighbor)

We have introduced multiple measures for comparing movies. How do we use these similarity functions to recommend movies? We can rephrase this problem: Given a user (the *target user*) who hasn't seen a certain movie (the *target movie*) yet, how would this user rate the movie after watching it?

So, the core of the problem is predicting a "missing" rating.

There are many possible approaches, but a commonly used method for recommending items is **K-nearest neighbor (KNN) regression**. That's a mouthful, but the intuition is simple: We decide on a number _K_ (for example, 5). Then, for the *target movie*, we look at the top _K_ (5) movies that are most similar. This group is called the *neighborhood*. Finally, we calculate the average rating given by the *target user* to the movies in the neighborhood.

The algorithm consists of two steps:

1. Select the *neighborhood* of the *target movie*. Select all movies for which:
    - the *target user* has seen the movie
    - the movie is very similar to the *target movie*
    
    
2. Calculate the weighted average of the selected movies. This means  the more similar a selected movie is to the *target movie*, the more it contributes to the average.

To work out these steps, we need a bit more data. To try out the algorithm, we'll manually add some data. We introduce two additional users (123 and 456) and one extra movie (4444):

In [ ]:
# Create some mock data
l = [[123, 2571, 5.0, 0], [123, 2710, 1.5, 0], [123, 4444, 4.0, 0], [456, 2571, 1.5, 0], 
     [456, 2710, 5.0, 0], [456, 4444, 1.0, 0], [182, 4444, 3.0, 0], [195, 4444, 2.0, 0], 
     [204, 4444, 4.0, 0], [376, 4444, 4.5, 0], [542, 4444, 4.5, 0], [123, 5555, 3.5, 0], 
     [456, 5555, 3.5, 0]]
extra_data = pd.DataFrame(data=l, columns = ratings.columns)
more_ratings = pd.concat([ratings, extra_data]).reset_index(drop=True)

display(more_ratings)

# Recompute utility matrix
utility_matrix2 = pivot_ratings(more_ratings)
display(utility_matrix2)

utility_matrix2_filled = fill_nan_mean(utility_matrix2)

# Recompute similarity matrix
centered_matrix = mean_center_columns(utility_matrix2_filled)
similarity = create_similarity_matrix_cosine(centered_matrix)
display(similarity)

As you can see in the utility matrix, the new users (123 and 456) have no rating for the movie Shrek (4306). We filled these with the average rating of the movie before calculating cosine similarity, but now we'll try to predict them in a better way.

To get a sense of this new data, let's again create a plot:

In [ ]:
# To see the difference
# plot_scatter_matrix(utility_matrix2.T.fillna(utility_matrix2.mean(axis=1)).T)
plot_scatter_matrix(utility_matrix2)

### Question 13

\[2 pts.\]

Looking at the plots above, which movie would you recommend to user 204? Explain your choice.

YOUR ANSWER HERE

### Question 14

\[1 pt.\]

We will first work on Step 1 of the algorithm:

1. Select the neighborhood of the target movie. Select all movies for which:
    - the target user has seen the movie
    - the movie is very similar to the target movie

We need to provide a definition of "very similar." For this, we have many choices. Often, the top $k$ most similar items are used, where the best value for $k$ is determined empirically. Note that it's important to only consider items for which the _cosine similarity_ is greater than 0.

Implement the function `select_neighborhood()` below. This function takes a `Series` with similarities, a `Series` with ratings, and the parameter `k` as input, and returns a `Series` with the top `k` `movieId`s and their corresponding similarities. For example, for user 123, movie 4306, and k = 10, the function should return the following `Series`:

    movieId
    2571    0.198517
    5555    0.335994
    Name: 4306, dtype: float64
    
Note: This output doesn't include movies 2710 and 4444. Movies 2710 and 4444 are missing because their similarity is smaller than 0, and 4306 is missing because user 123 hasn't seen it yet. Moreover, the output contains only 2 items instead of 10. This is because there are only 2 movies that have a similarity greater than 0 with movie 4306.

> Tip: If you look at the function call to `select_neighborhood`, you'll notice that we're using `utility_matrix2` there and not `utility_matrix2_filled`. Why is that?

In [ ]:
def select_neighborhood(similarities: pd.Series, ratings: pd.Series, k: int) -> pd.Series:
    """
    Selects the k most similar items from all items with similarity > 0.
    """
    # TODO
    
    
neighborhood = select_neighborhood(similarity[4306], utility_matrix2[123], 10)
display(neighborhood)

### Check

In [ ]:
answers.test_14(select_neighborhood, similarity, utility_matrix2)

### Question 15

\[2 pts.\]

Now, only Step 2 of the algorithm remains:

2. Calculate the weighted average of the selected movies. For this, the more a movie is similar to the _target movie_, the more it will contribute to the average.

In the example above, the neighborhood contains similarities $0.73$ and $0.58$ for movies 2571 and 4444, respectively. The ratings given by user 123 for these movies are $5.0$ and $4.0$. The weighted average is then:
$$
\textrm{predicted rating} = \frac{5.0 * 0.73 + 4.0 * 0.58}{0.73 + 0.58} \approx 4.56
$$

The formal definition (with $r_i$ being the rating and $s_i$ being the similarity for movie $i$):
$$
\textrm{predicted rating} = \frac{\sum_{\textrm{i in neighborhood}} r_i * s_i}{\sum_{\textrm{i in neighborhood}}s_i}
$$

Implement the final step of the item-based collaborative filtering algorithm below in the `weighted_mean()` function.

In [ ]:
def weighted_mean(neighborhood: pd.Series, ratings: pd.Series) -> float:  
    # TODO
    
    
neighborhood1 = select_neighborhood(similarity[4306], utility_matrix2[123], 10)
neighborhood2 = select_neighborhood(similarity[4306], utility_matrix2[456], 10)
prediction1 = weighted_mean(neighborhood1, utility_matrix2[123])
prediction2 = weighted_mean(neighborhood2, utility_matrix2[456])

print(f'User 123 predicted rating for Shrek {prediction1:.2f}')
print(f'User 456 predicted rating for Shrek {prediction2:.2f}')  

### Check

In [ ]:
answers.test_15(select_neighborhood, similarity, utility_matrix2, weighted_mean)

### Question 16

\[1 pt.\]

Conclusion: Would you recommend Shrek to user 123? And to user 456?

YOUR ANSWER HERE

# Part 2: User-Based Filtering

So far, we've been looking at item-based filtering. In this approach, we predict ratings based on the similarity of items (movies). The primary counterpart of item-based filtering is *user-based filtering*. This is the collaborative filtering equivalent of asking someone with a similar taste in movies for advice.

The process of user-based filtering is almost identical to item-based filtering. We just reverse the roles:

- Instead of a utility matrix with users on the horizontal axis and movies on the vertical axis, we switch the axes.
- Instead of creating a similarity matrix for movies, we create a similarity matrix for users.
- Instead of selecting a neighborhood of the _target movie_, we select a neighborhood of the _target user_.
- Instead of calculating a weighted average of ratings from similar movies, we calculate a weighted average of ratings from similar users.

For the questions below, we'll work with _additional rating data_.

Tip: For the questions below, you won't need to write much code. **Use the functions you've written in the previous part as much as possible.** If you're writing more than a few lines of code in a cell, you're likely overcomplicating it.

In [ ]:
display(more_ratings)

### Question 17

\[1 pt.\]

Create a new utility matrix and save it as `utility_matrix3`. Use the users as rows and movies as columns.

Also, create a version of `utility_matrix3` with filled NaN values and name it `utility_matrix3_filled`. Make sure to fill in the average rating _for the movie_.

Tip: You can swap the rows and columns of a `DataFrame` `df` by using `df.T`.

In [ ]:
# TODO


display(utility_matrix3)
display(utility_matrix3_filled)

### Check

In [ ]:
answers.test_17(utility_matrix3)

### Question 18

\[1 pt.\]

Create a scatter plot matrix of the utility matrix.

In [ ]:
# TODO


### Question 19

\[1 pt.\]

Create a mean-centered utility matrix named `centered_utility_matrix`. Note: This is the only step where the roles are not swapped. We still want to use the average rating of the _users_ as an offset, not that of the movies. The goal is not to measure how a user deviates from the average rating of a movie, but rather how the user deviates from their average rating for a movie. In other words, you need to center the rows here instead of the columns.

In [ ]:
# TODO


display(centered_utility_matrix)

### Check

In [ ]:
answers.test_19(centered_utility_matrix)

### Question 20

\[1 pt.\]

Calculate the adjusted cosine similarity matrix for the users and store it in a variable named `similarity`.

In [ ]:
# TODO


display(similarity)

### Check

In [ ]:
answers.test_20(similarity)

### Question 21

\[1 pt.\]

Calculate the neighborhood for user 123 and 456 for movie 4306. Store the neighborhood for user 123 in a variable named `neighborhood1` and for user 456 in a variable named `neighborhood2`.

In [ ]:
# TODO


print(neighborhood1) # print the neighborhood for user 123
print(neighborhood2) # print the neighborhood for user 456

### Check

In [ ]:
answers.test_21(neighborhood1, neighborhood2)

### Question 22

\[1 pt.\]

Calculate the weighted average for the neighborhoods of user 123 and 456 (still using the ratings for movie 4306).

In [ ]:
# TODO


print(f'User 123 predicted rating for Shrek {prediction1:.2f}')
print(f'User 456 predicted rating for Shrek {prediction2:.2f}') 

### Check

In [ ]:
answers.test_22(prediction1, prediction2)

### Question 23

\[3 pts.\]

Comparing this user-based prediction with the item-based prediction, which one do you find better? Look at the initial data and the predictions and explain your findings in a few sentences. (In the next module, we'll explore how to evaluate the quality of the approach.)

YOUR ANSWER HERE

Which algorithm is better? Maybe you can make a judgment about that based on this specific dataset. But in general, how well does a specific recommender system work? How do we even define "good"? We will explore these questions in the next module.